# 13. プリ・ポストデザインデータの分析

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from scipy import stats
%matplotlib inline
from itertools import cycle
import pyper

sns.set(style='ticks')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

## 13.1 プリポストデザイン
1. ポストテストの値を従属変数、プリテストの値を統制変数（共変量）として、群の効果に関する共分散分析を行う。
2. 「ポストテストの値-プリテストの値」である変化量に関して、t検定によって群の効果について吟味する（これは、変化量に関して、対応のないt検定（独立な2群のt検定）を実行するということ）。
3. 測定時期（プリ、ポスト）を1つの要因として、群×測定時期の2要因分散分析を行い、両者の交互作用効果について吟味する（これは、群が被験者間、測定時期が被験者内の似要因混合計画の分散分析を実行するということ）。

1,2が妥当な方法。3は不適切であると述べている。

## 13.2

In [2]:
df = pd.read_csv('prepost.csv')
df

,group,pre,post
0,control,35,40
1,control,40,23
2,control,39,28
3,control,18,46
4,control,22,47
5,control,41,42
6,control,41,27
7,control,47,33
8,exp,35,36
9,exp,41,71


In [32]:
# 何かが違う
import statsmodels.api as sm
from statsmodels.formula.api import ols
model = ols('post~pre*group', df).fit()
aov_table = sm.stats.anova_lm(model, typ=2)
print(aov_table)

                sum_sq    df          F    PR(>F)
group      1820.385275   1.0  15.997813  0.001763
pre         405.815208   1.0   3.566364  0.083368
pre:group     0.084205   1.0   0.000740  0.978745
Residual   1365.475587  12.0        NaN       NaN


In [33]:
model = ols('post~pre+group', df).fit()
aov_table = sm.stats.anova_lm(model, typ=2)
print(aov_table)

               sum_sq    df          F    PR(>F)
group     1820.385275   1.0  17.329896  0.001114
pre        405.815208   1.0   3.863322  0.071085
Residual  1365.559792  13.0        NaN       NaN


In [9]:
df['change'] = df['post'] - df['pre']
df

,group,pre,post,change
0,control,35,40,5
1,control,40,23,-17
2,control,39,28,-11
3,control,18,46,28
4,control,22,47,25
5,control,41,42,1
6,control,41,27,-14
7,control,47,33,-14
8,exp,35,36,1
9,exp,41,71,30


In [24]:
# 分散の等質性
stats.bartlett(df['change'].values, df['group'].values)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
# 分散の等質性
change = df['change'].values
group = df['group'].values
# group = np.where(df['group'].values, df['group']=='control', 1)
stats.bartlett(change, group)

In [25]:
# p値が5%以上(等質性がある)
stats.ttest_ind(df['group'].values, df['change'].values, equal_var=True)

TypeError: unsupported operand type(s) for /: 'str' and 'int'